In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
df=pd.read_csv('InternetAds_norm_19.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,...,att1550,att1551,att1552,att1553,att1554,att1555,att1556,att1557,att1558,outlier
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
df.shape

(3264, 1556)

In [4]:
# df to values
df = df.values

In [5]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/InternetAds.json")  
gc = GCForest(config)

In [17]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,1555]
X = df[:,0:1555]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
# count class
len(np.unique(y_train))

2

In [8]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 09:00:31,066][cascade_classifier.fit_transform] X_groups_train.shape=[(2284, 1555)],y_train.shape=(2284,),X_groups_test.shape=[(980, 1555)],y_test.shape=(980,)
[ 2018-04-25 09:00:31,089][cascade_classifier.fit_transform] group_dims=[1555]
[ 2018-04-25 09:00:31,091][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:00:31,093][cascade_classifier.fit_transform] group_ends=[1555]
[ 2018-04-25 09:00:31,094][cascade_classifier.fit_transform] X_train.shape=(2284, 1555),X_test.shape=(980, 1555)
[ 2018-04-25 09:00:31,116][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2284, 1555), X_cur_test.shape=(980, 1555)
[ 2018-04-25 09:00:31,909][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=93.89%
[ 2018-04-25 09:00:32,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.51%
[ 2018-04-25 09:00:33,604][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - e

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 09:00:39,517][cascade_classifier.transform] X_groups_test.shape=[(980, 1555)]
[ 2018-04-25 09:00:39,527][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-25 09:00:39,528][cascade_classifier.transform] X_test.shape=(980, 1555)
[ 2018-04-25 09:00:39,531][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(980, 1555)


Test Accuracy of ExtratTrees = 97.653061 %


In [10]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 09:00:41,052][cascade_classifier.transform] X_groups_test.shape=[(980, 1555)]
[ 2018-04-25 09:00:41,057][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-25 09:00:41,059][cascade_classifier.transform] X_test.shape=(980, 1555)
[ 2018-04-25 09:00:41,063][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(980, 1555)


In [11]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [12]:
X_test_new.shape

(980, 1557)

In [13]:
X_train_new.shape

(2284, 1557)

In [14]:
# Cascade ExtraTrees
config = load_json("./examples/InternetAdsgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 09:00:42,757][cascade_classifier.fit_transform] X_groups_train.shape=[(2284, 1557)],y_train.shape=(2284,),X_groups_test.shape=[(980, 1557)],y_test.shape=(980,)
[ 2018-04-25 09:00:42,779][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-25 09:00:42,780][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:00:42,782][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-25 09:00:42,783][cascade_classifier.fit_transform] X_train.shape=(2284, 1557),X_test.shape=(980, 1557)
[ 2018-04-25 09:00:42,804][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2284, 1557), X_cur_test.shape=(980, 1557)
[ 2018-04-25 09:00:43,392][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.63%
[ 2018-04-25 09:00:44,168][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.51%
[ 2018-04-25 09:00:44,897][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - e

[ 2018-04-25 09:01:17,525][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=96.51%
[ 2018-04-25 09:01:18,274][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=96.07%
[ 2018-04-25 09:01:19,190][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=95.63%
[ 2018-04-25 09:01:20,044][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.12%
[ 2018-04-25 09:01:20,836][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=96.93%
[ 2018-04-25 09:01:21,758][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=97.37%
[ 2018-04-25 09:01:22,594][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=96.93%
[ 2018-04-25 09:01:23,474][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=96.93%


In [15]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [16]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/InternetAdsgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 09:01:32,525][cascade_classifier.fit_transform] X_groups_train.shape=[(2284, 1557)],y_train.shape=(2284,),X_groups_test.shape=[(980, 1557)],y_test.shape=(980,)
[ 2018-04-25 09:01:32,536][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-25 09:01:32,537][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:01:32,539][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-25 09:01:32,540][cascade_classifier.fit_transform] X_train.shape=(2284, 1557),X_test.shape=(980, 1557)
[ 2018-04-25 09:01:32,559][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2284, 1557), X_cur_test.shape=(980, 1557)
[ 2018-04-25 09:01:33,263][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=94.32%
[ 2018-04-25 09:01:34,019][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.07%
[ 2018-04-25 09:01:34,835][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - e

[ 2018-04-25 09:02:08,640][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=96.94%
[ 2018-04-25 09:02:09,390][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=95.63%
[ 2018-04-25 09:02:10,156][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=96.07%
[ 2018-04-25 09:02:11,065][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=98.25%
[ 2018-04-25 09:02:12,048][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=96.93%
[ 2018-04-25 09:02:13,047][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.68%
[ 2018-04-25 09:02:13,904][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=96.93%
[ 2018-04-25 09:02:14,787][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=97.37%


[ 2018-04-25 09:02:48,001][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_7.predict)=97.37%
[ 2018-04-25 09:02:48,763][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_8.predict)=96.05%
[ 2018-04-25 09:02:49,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_9.predict)=94.74%
[ 2018-04-25 09:02:49,766][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_cv.predict)=97.07%
[ 2018-04-25 09:02:49,768][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.test.predict)=97.55%
[ 2018-04-25 09:02:49,770][cascade_classifier.calc_accuracy] Accuracy(layer_8 - train.classifier_average)=97.07%
[ 2018-04-25 09:02:49,772][cascade_classifier.calc_accuracy] Accuracy(layer_8 - test.classifier_average)=97.55%
[ 2018-04-25 09:02:49,789][cascade_classifier.fit_transform] [layer=9] look_indexs=[0], X_cur_train.shape=(2284, 1559), X_cur_test.shape=(980, 1559)
[ 2

[ 2018-04-25 09:03:21,842][cascade_classifier.calc_accuracy] Accuracy(layer_12 - test.classifier_average)=97.55%
[ 2018-04-25 09:03:21,844][cascade_classifier.fit_transform] [Result][Optimal Level Detected] opt_layer_num=10, accuracy_train=97.42%, accuracy_test=97.65%
[ 2018-04-25 09:03:21,866][cascade_classifier.transform] X_groups_test.shape=[(980, 1557)]
[ 2018-04-25 09:03:21,870][cascade_classifier.transform] group_dims=[1557]
[ 2018-04-25 09:03:21,871][cascade_classifier.transform] X_test.shape=(980, 1557)
[ 2018-04-25 09:03:21,875][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(980, 1557)
[ 2018-04-25 09:03:23,578][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(980, 1559)
[ 2018-04-25 09:03:25,153][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(980, 1559)
[ 2018-04-25 09:03:26,730][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(980, 1559)
[ 2018-04-25 09:03:28,295][casc

('Test Accuracy of Extratrees = 97.653061 %', 0, 'fois')


...

('Test Accuracy of Extratrees = 97.755102 %', 18, 'fois')


[ 2018-04-25 09:23:43,861][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=94.32%
[ 2018-04-25 09:23:44,631][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.07%
[ 2018-04-25 09:23:45,503][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.69%
[ 2018-04-25 09:23:46,384][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.38%
[ 2018-04-25 09:23:47,218][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.93%
[ 2018-04-25 09:23:47,934][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.37%
[ 2018-04-25 09:23:48,779][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.49%
[ 2018-04-25 09:23:49,678][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.93%


[ 2018-04-25 09:24:20,835][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.68%
[ 2018-04-25 09:24:21,698][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=96.93%
[ 2018-04-25 09:24:22,429][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=97.37%
[ 2018-04-25 09:24:23,173][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=96.93%
[ 2018-04-25 09:24:23,305][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.02%
[ 2018-04-25 09:24:23,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.76%
[ 2018-04-25 09:24:23,309][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.02%
[ 2018-04-25 09:24:23,311][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.76%
[ 2018-04-25 09:24:23,340][

[ 2018-04-25 09:24:53,987][cascade_classifier.calc_accuracy] Accuracy(layer_8 - train.classifier_average)=96.76%
[ 2018-04-25 09:24:53,989][cascade_classifier.calc_accuracy] Accuracy(layer_8 - test.classifier_average)=97.24%
[ 2018-04-25 09:24:54,004][cascade_classifier.fit_transform] [layer=9] look_indexs=[0], X_cur_train.shape=(2284, 1559), X_cur_test.shape=(980, 1559)
[ 2018-04-25 09:24:54,594][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_0.predict)=97.38%
[ 2018-04-25 09:24:55,354][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_1.predict)=96.51%
[ 2018-04-25 09:24:56,109][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_2.predict)=97.38%
[ 2018-04-25 09:24:56,895][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_3.predict)=96.07%
[ 2018-04-25 09:24:57,665][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_4.predict)=97.37%
[

('Test Accuracy of Extratrees = 97.857143 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 97.857143 %', 19, 'fois')